In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from loguru import logger


In [2]:
data = {}
path = r"G:\Mi unidad\walknet_datalake\sources\crtm\level0"
for n in [4,5,6,8,10]:
    data.update({n:pd.read_csv("{path}\M{n}_Estaciones.csv".format(path = path, n=str(n)))})

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\katul\AppData\Local\Temp\ipykernel_16740\2512940626.py:4: SyntaxWarning: invalid escape sequence '\M'
  data.update({n:pd.read_csv("{path}\M{n}_Estaciones.csv".format(path = path, n=str(n)))})


In [3]:
# Function to apply to each element
def process_element(x, m):
    if isinstance(x, list):  # Check if the element is a list
        return [m+"|"+str(i) for i in x]  # For example, multiply each element in the list by 2
    else:
        return m+"|"+str(x)  # For single values, just multiply by 2

In [4]:
# Load GTFS data into DataFrames
stops = pd.read_csv(path+r'\stops.txt')
trips = pd.read_csv(path+r'\trips.txt')
stop_times = pd.read_csv(path+r'\stop_times.txt')
routes = pd.read_csv(path+r'\routes.txt')

# Merge stop_times with stops to associate each stop with trips
stop_times_stops = pd.merge(stop_times, stops, on='stop_id')

# Merge with trips to associate each trip with route
stop_times_stops_trips = pd.merge(stop_times_stops, trips, on='trip_id')

# Merge with routes to associate trips with route information
stop_times_stops_trips_routes = pd.merge(stop_times_stops_trips, routes, on='route_id')

# Select relevant columns including x and y
stops_lines = stop_times_stops_trips_routes[['stop_id', 'stop_name', 'stop_lon', 'stop_lat', 'route_short_name']]

# Aggregate unique routes for each stop
stops_lines_agg = stops_lines.groupby(['stop_id', 'stop_name', 'stop_lon', 'stop_lat',])['route_short_name'].agg(lambda x: list(set(x))).reset_index()

# Display the resulting dataframe
stops_lines_agg['geometry'] = stops_lines_agg.apply(lambda x: Point(x['stop_lon'],x['stop_lat']),axis=1)
stops_lines_agg.rename(columns={'stop_id':'id','route_short_name':'lines'},inplace=True)
stops_lines_agg['mode'] = 'Bus'
stops_lines_agg = stops_lines_agg[['id','mode','lines','geometry']]
stops_lines_agg = gpd.GeoDataFrame(stops_lines_agg, geometry='geometry',crs=4326)
stops_lines_agg = stops_lines_agg.to_crs(25830)


In [5]:
#stops_lines_agg['lines'] = stops_lines_agg.apply(lambda r: process_element(r['lines'],r['mode'][0]),axis=1)

In [6]:
#stops_lines_agg['lines'] = stops_lines_agg['lines'].apply(lambda x: "["+','.join(str(e) for e in x)+"]")

In [7]:
t = {4:'Train',5:'Train',6:'Bus',9:'Bus',8:'Bus',10:'Train'}

In [8]:
for k,v in data.items():
    
    data[k] = data[k].rename(columns={'IDESTACION':'id','LINEAS':'lines'})
    data[k]['geometry'] = data[k]['geometry'] = data[k].apply(lambda x: Point(x['X'],x['Y']),axis=1)
    data[k]['mode'] = t[k]
    data[k].dropna(subset='lines',inplace=True)
    #data[k]['lines'] = data[k].apply(lambda r: process_element(r['lines'],r['mode'][0]),axis=1)
    
    data[k]['lines'] = data[k]['lines'].astype(str)
    #data[k]['lines'] = "["+data[k]['lines']+"]"
    
    data[k]['lines'] = data[k]['lines'].str.split(",")
    data[k] = data[k][['id','mode','lines','geometry']]
    data[k] = gpd.GeoDataFrame(data[k],geometry='geometry',crs=25830)


In [9]:
data.update({9:stops_lines_agg})

In [10]:
def json_data(data: pd.DataFrame, columns, idx: str) -> pd.Series:
    """
    Create JSON data from a DataFrame.

    Parameters:
        data (pd.DataFrame): The DataFrame to be converted.
        columns (List[str]): The columns to include in the JSON.
        idx (str): The column to use as the JSON index.

    Returns:
        A pandas Series with the JSON data.
    """
    try:
        d = pd.Series(data.set_index(idx)[[c for c in columns if c in data.columns]].to_dict(orient='records'))
        d = d.astype(str).str.replace("'",'"')
        logger.info('Making JSON DATA columns...')
        return d
    except Exception as e:
        logger.error(f"Failed to create JSON data. Error: {e}")
        return pd.Series()

In [11]:
gdf = pd.concat(data.values())

In [12]:
gdf['id_class'] = 'pois'
gdf['category'] = 'transportation'
gdf['provider'] = 99
gdf['geometry'] = gdf['geometry'].to_wkt()

C:\Users\katul\AppData\Local\Temp\ipykernel_16740\1433239317.py:4: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].to_wkt()


In [13]:
gdf

,id,mode,lines,geometry,id_class,category,provider
0,4_153,Train,[7],POINT (438581 4482705),pois,transportation,99
1,4_183,Train,[9],POINT (453645 4468818),pois,transportation,99
2,4_38,Train,"[2, 3, 10]",POINT (440100 4475360),pois,transportation,99
3,4_199,Train,[10],POINT (437699 4474444),pois,transportation,99
4,4_200,Train,[10],POINT (436208 4473507),pois,transportation,99
...,...,...,...,...,...,...,...
6606,par_8_99982,Bus,[541],POINT (398823.99088 4455301.927264),pois,transportation,99
6607,par_8_99983,Bus,[548],POINT (394811.989795 4458052.037822),pois,transportation,99
6608,par_8_99984,Bus,[548],POINT (394795.004572 4458043.804662),pois,transportation,99
6609,par_8_99986,Bus,"[546, 545]",POINT (383527.98627 4460740.141425),pois,transportation,99


In [14]:
import json
import ast

# Function to extract the desired values
def extract_first_letter_mode_and_lines(row):
    
    #obj = json.loads(row)  # Convert JSON string to dictionary
    mode_first_letter = row['mode'][0]  # Get the first letter of "mode"

#    lines_list = ast.literal_eval(obj['lines'])  # Convert the string to a list
    #lines_str = row['lines'].replace('[', '').replace(']', '')
    #lines_list = [line.strip() for line in lines_str.split(',')]
    #print(row['lines'])
    return [f"{mode_first_letter}{line.strip()}" for line in row['lines']]

# Apply the function to each row in the 'column'
gdf['mode_lines'] = gdf.apply(extract_first_letter_mode_and_lines,axis=1)



In [15]:
gdf

,id,mode,lines,geometry,id_class,category,provider,mode_lines
0,4_153,Train,[7],POINT (438581 4482705),pois,transportation,99,[T7]
1,4_183,Train,[9],POINT (453645 4468818),pois,transportation,99,[T9]
2,4_38,Train,"[2, 3, 10]",POINT (440100 4475360),pois,transportation,99,"[T2, T3, T10]"
3,4_199,Train,[10],POINT (437699 4474444),pois,transportation,99,[T10]
4,4_200,Train,[10],POINT (436208 4473507),pois,transportation,99,[T10]
...,...,...,...,...,...,...,...,...
6606,par_8_99982,Bus,[541],POINT (398823.99088 4455301.927264),pois,transportation,99,[B541]
6607,par_8_99983,Bus,[548],POINT (394811.989795 4458052.037822),pois,transportation,99,[B548]
6608,par_8_99984,Bus,[548],POINT (394795.004572 4458043.804662),pois,transportation,99,[B548]
6609,par_8_99986,Bus,"[546, 545]",POINT (383527.98627 4460740.141425),pois,transportation,99,"[B546, B545]"


In [16]:


#gdf.set_index('id',inplace=True,drop=False)
gdf['data'] = pd.Series(gdf.set_index('id')[['mode','mode_lines']].to_dict(orient='records')).astype(str).str.replace("'",'"')
#gdf['data'] = json_data(gdf, ['mode','lines'], 'id' )



In [17]:
gdf
#gdf['data'] = json_data(gdf, ['mode','lines'], 'id' )


,id,mode,lines,geometry,id_class,category,provider,mode_lines,data
0,4_153,Train,[7],POINT (438581 4482705),pois,transportation,99,[T7],"{""mode"": ""Train"", ""mode_lines"": [""T7""]}"
1,4_183,Train,[9],POINT (453645 4468818),pois,transportation,99,[T9],"{""mode"": ""Train"", ""mode_lines"": [""T9""]}"
2,4_38,Train,"[2, 3, 10]",POINT (440100 4475360),pois,transportation,99,"[T2, T3, T10]","{""mode"": ""Train"", ""mode_lines"": [""T2"", ""T3"", ""..."
3,4_199,Train,[10],POINT (437699 4474444),pois,transportation,99,[T10],"{""mode"": ""Train"", ""mode_lines"": [""T10""]}"
4,4_200,Train,[10],POINT (436208 4473507),pois,transportation,99,[T10],"{""mode"": ""Train"", ""mode_lines"": [""T10""]}"
...,...,...,...,...,...,...,...,...,...
6606,par_8_99982,Bus,[541],POINT (398823.99088 4455301.927264),pois,transportation,99,[B541],"{""mode"": ""Bus"", ""mode_lines"": [""B250"", ""BN202""]}"
6607,par_8_99983,Bus,[548],POINT (394811.989795 4458052.037822),pois,transportation,99,[B548],"{""mode"": ""Bus"", ""mode_lines"": [""B251"", ""B255""]}"
6608,par_8_99984,Bus,[548],POINT (394795.004572 4458043.804662),pois,transportation,99,[B548],"{""mode"": ""Bus"", ""mode_lines"": [""B523"", ""BN503""]}"
6609,par_8_99986,Bus,"[546, 545]",POINT (383527.98627 4460740.141425),pois,transportation,99,"[B546, B545]","{""mode"": ""Bus"", ""mode_lines"": [""B523"", ""BN503""]}"


In [18]:
def update_lines_in_json(row):
    obj = json.loads(row)  # Parse the original JSON string into a dictionary
    return obj

# Apply the function to update each row
#gdf['lines'] = gdf['mode_lines']
#gdf['lines'] = gdf['lines'].astype(str)
#gdf['lines'] = "["+gdf['lines']+"]"

#.astype(str).str.replace("'",'"')


gdf = gdf[['id','id_class','category','provider','data','geometry']]

In [19]:
#gdf['mode_lines'].apply(lambda x: str(x)[0])

In [20]:
#gdf['data'] =pd.Series(gdf.set_index('id')[['mode','lines']].to_dict(orient='records')).astype(str).str.replace("'",'"')

In [21]:
gdf.to_csv(r"G:\Mi unidad\walknet_datalake\sources\crtm\level2\level2_crtm.csv",sep=";",index=False)

In [22]:
gdf

,id,id_class,category,provider,data,geometry
0,4_153,pois,transportation,99,"{""mode"": ""Train"", ""mode_lines"": [""T7""]}",POINT (438581 4482705)
1,4_183,pois,transportation,99,"{""mode"": ""Train"", ""mode_lines"": [""T9""]}",POINT (453645 4468818)
2,4_38,pois,transportation,99,"{""mode"": ""Train"", ""mode_lines"": [""T2"", ""T3"", ""...",POINT (440100 4475360)
3,4_199,pois,transportation,99,"{""mode"": ""Train"", ""mode_lines"": [""T10""]}",POINT (437699 4474444)
4,4_200,pois,transportation,99,"{""mode"": ""Train"", ""mode_lines"": [""T10""]}",POINT (436208 4473507)
...,...,...,...,...,...,...
6606,par_8_99982,pois,transportation,99,"{""mode"": ""Bus"", ""mode_lines"": [""B250"", ""BN202""]}",POINT (398823.99088 4455301.927264)
6607,par_8_99983,pois,transportation,99,"{""mode"": ""Bus"", ""mode_lines"": [""B251"", ""B255""]}",POINT (394811.989795 4458052.037822)
6608,par_8_99984,pois,transportation,99,"{""mode"": ""Bus"", ""mode_lines"": [""B523"", ""BN503""]}",POINT (394795.004572 4458043.804662)
6609,par_8_99986,pois,transportation,99,"{""mode"": ""Bus"", ""mode_lines"": [""B523"", ""BN503""]}",POINT (383527.98627 4460740.141425)
